In [32]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


from transformers import *
import os
import sys
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import numpy as np
import re
import pickle
import time
import pandas as pd
from pathlib import Path
import random
# from torch.utils.tensorboard import SummaryWriter

# 1. Test data Embedding 

In [33]:
# load datasets
df = pd.read_csv('./data/hackathon_test_for_user.csv', encoding='cp949')
# df = pd.read_excel('./data_phase2/test_data.xlsx', sheet_name='Sheet1', index_col=0)
df['Age_float'] = df['Age']
# df = df.sort_values(by=['User_ID'])

# df['Gender'] = df['Gender'].map({'male': 0, 'female': 1})
df = pd.get_dummies(df, columns=['Age'], prefix='cat')

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3600 entries, 1 to 12749
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   User_ID             3600 non-null   object
 1   Gender              3600 non-null   object
 2   Q_number            3600 non-null   object
 3   Age_float           3600 non-null   object
 4   cat_20              3600 non-null   object
 5   cat_30              3600 non-null   object
 6   cat_40              3600 non-null   object
 7   Long_Answer_concat  3600 non-null   object
dtypes: object(8)
memory usage: 253.1+ KB


In [35]:
# # now concatenate the "Long_Answer" of k similar user_ids

# new_columns = [col for col in df.columns if col not in ['Short_Answer', 'Long_Answer']]
# new_df = pd.DataFrame(columns=new_columns)

# num_samples = df.shape[0]
# k_user_ids = 2
# # print(df['Long_Answer'])
# for i in range(0, num_samples, k_user_ids):
    
#     user_ids = df.iloc[i:i+k_user_ids]['User_ID'].to_list()
    
#     # check if all the user_ids are the same
#     if len(set(user_ids)) == 1:
        
#         # concatenate the "Long_Answer" of k similar user_ids

#         long_answer_concat = df.iloc[i:i+k_user_ids]['Long_Answer'].to_list()
        
#         concat_str = ' 그리고 '.join(long_answer_concat)
        
#         # print(concat_str)
        
#         # since the "User_ID", and all other elements are the same, we can just take the first one
#         new_row = df.iloc[i:i+k_user_ids].head(1)[new_columns]
#         new_row['Long_Answer_concat'] = concat_str
        
#         new_df = pd.concat([new_df, new_row], axis=0)
    

# # print(new_df)
# # print(df.shape)
# assert new_df.shape[0] == df.shape[0] // k_user_ids
# assert new_df.shape[1] == df.shape[1] - 1

# df = new_df

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3600 entries, 1 to 12749
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   User_ID             3600 non-null   object
 1   Gender              3600 non-null   object
 2   Q_number            3600 non-null   object
 3   Age_float           3600 non-null   object
 4   cat_20              3600 non-null   object
 5   cat_30              3600 non-null   object
 6   cat_40              3600 non-null   object
 7   Long_Answer_concat  3600 non-null   object
dtypes: object(8)
memory usage: 253.1+ KB


In [37]:
# model_name = 'kykim/bert-kor-base'
# model_name = 'monologg/kobigbird-bert-base'
# model_name = 'beomi/kcbert-base'
model_name = 'snunlp/KR-BERT-char16424'
# model_name = 'klue/bert-base'
# model_name = 'klue/roberta-base'

In [38]:
# load pretrained model
def get_model(model_name):
    # * Model          | Tokenizer          | Pretrained weights shortcut
    # MODEL=(DistilBertModel, DistilBertTokenizer, 'distilbert-base-uncased')
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    n_hl = model.config.num_hidden_layers
    # embed_dim = model.config.embedding_size
    embed_dim = 768
    return model, tokenizer, n_hl, embed_dim

In [39]:
model, tokenizer, n_hl, embed_dim = get_model(model_name)

loading configuration file config.json from cache at /home/kaleb/.cache/huggingface/hub/models--snunlp--KR-BERT-char16424/snapshots/47521960ac7595c5d2ed643f7a9dab9b0efcf58d/config.json
Model config BertConfig {
  "_name_or_path": "snunlp/KR-BERT-char16424",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 16424
}

loading file vocab.txt from cache at /home/kaleb/.cache/huggingface/hub/models--snunlp--KR-BERT-char16424/snapshots/47521960ac7595c5d2ed643f7a9dab9b0efcf58d/vocab.txt
loading file tokenizer.json from cache at None
loading file add

In [40]:
# df['One_word_answer'] = df['Answer'].apply(lambda x: (x.split('>')[0])[x.index('<')+1:])
# # df['One_word_answer'] = df['One_word_answer'].astype('category').cat.codes
# # df['One_word_answer'].unique()

# df['One_word_ans_enc'] = df['One_word_answer'].apply(lambda x : 1 if x.startswith('중') else (2 if x.startswith('아') or x.startswith('어') else 0))

# df['Answer_rep'] = df['Answer'].apply(lambda x: x.replace(f"<{(x.split('>')[0])[x.index('<')+1:]}>", ''))

# # df = pd.get_dummies(df, columns=['One_word_ans_enc'], prefix='is')
# print(df['One_word_answer'].unique())
# # df['One_word_answer'].unique() # array(['아니다', '중립', '그렇다', '아니오', '중간', '보통', '아니요']
# # df

In [41]:
# import re
# from konlpy.tag import Okt
# from hanspell import spell_checker
# import json
# from num2words import num2words
# from kss import split_sentences

# def sentence_segmentation(text):
#     sentences = split_sentences(text)
#     return sentences

# def pos_tagging(text):
#     okt = Okt()
#     tagged = okt.pos(text)
#     return tagged



# def preprocess_text(text, apply_ngrams=False, ngram_size=1, apply_replacements=True, apply_normalization=True, remove_stopwords=False):
#     # Check and correct spelling
#     # checked_spelling = spell_checker.check(text)
#     # text = checked_spelling.checked

#     # Remove special characters using regular expressions
#     # text = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9\s]", " ", text)

#     # Initialize the Okt tokenizer
#     okt = Okt()

#     # Tokenize the text into words
#     # words = okt.morphs(text, stem=False)
#     words = text.split()
#     # print(words)

#     # if apply_ner:
#     #     # Perform named entity recognition
#     #     entities = okt.nouns(text)
#     #     words = ["[NER]" if word in entities else word for word in words]

#     if apply_replacements:
#         # Replace specific words or phrases (example)
#         replacements = {"아니다": "아닙니다", "그렇다": "그렇습니다", "이다": "입니다"}
#         words = [replacements.get(word, word) for word in words]

#     if apply_normalization:
#         # Perform text normalization 
#         words = [num2words(word) if word.isdigit() else word for word in words]
#         words = [okt.normalize(word) for word in words]
#         # words = [normalization_map.get(word, word) for word in words]

#     # Remove stopwords
#     if remove_stopwords:
#         stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', '다']
#         words = [word for word in words if word not in stopwords]

#     # Remove short and long words
#     # words = [word for word in words if 3 <= len(word) <= 30]

#     if apply_ngrams:
#         # Generate N-grams
#         ngrams = zip(*[words[i:] for i in range(ngram_size)])
#         ngrams = [" ".join(ngram) for ngram in ngrams]
#         words.extend(ngrams)

#     return " ".join(words)




In [42]:
# df['preprocessed_ans'] = df['Answer_rep'].apply(preprocess_text)
df['Answer_rep'] = df['Answer'].apply(lambda x: x.replace(f"<{(x.split('>')[0])[x.index('<')+1:]}>", ''))

In [43]:
# if there are any columns that you wish to tokenize with bert, add them here
# if you are eliminating the one-word-short answer -> please modify Answer to Answer_rep
# tensor = tokenizer(df['Long_Answer_concat'].to_list(), max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)
tensor = tokenizer(df['Answer_rep'].to_list(), max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)

/home/kaleb/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2372: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [44]:
class MyMapDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, idx):
        data = {k:v[idx] for k,v in self.data.items()}
        return data

In [45]:
ds = MyMapDataset(tensor)
dl = DataLoader(ds, batch_size=32, shuffle=False)

In [46]:
def forward(model, dl, device=0):
    pooled = []
    # hidden = []
    model.cuda(device)
    model.eval()
    for data in dl:
        data = {k:v.cuda(device) for k,v in data.items()}
        with torch.no_grad():
            output = model(**data, output_hidden_states=False)
        p = output.pooler_output#, output.hidden_states
        pooled.append(p) # pooler output
        # hidden.append(h[-1][:,0,:]) # only [CLS] token embedding 
    return torch.cat(pooled)#, torch.cat(hidden)

In [47]:
result = forward(model, dl, device=0)

In [48]:
# result[0].shape, result[1].shape

In [49]:
# import numpy as np
# from collections import Counter
# from cuml.metrics import accuracy_score, roc_auc_score
# # from sklearn.metrics import roc_auc_score, 

# class CuMLVotingClassifier:
#     def __init__(self, estimators, voting='hard'):
#         self.estimators = estimators
#         self.n_estimators = len(estimators)
#         self.voting = voting

#     def fit(self, X, y):
#         for _, estimator in self.estimators:
#             estimator.fit(X, y)

#     def predict(self, X):
#         if self.voting == 'hard':
#             predictions = np.zeros((X.shape[0], self.n_estimators), dtype=np.int32)
#             for i, (_, estimator) in enumerate(self.estimators):
#                 predictions[:, i] = estimator.predict(X)
#             majority_vote = np.apply_along_axis(lambda x: Counter(x).most_common(1)[0][0], axis=1, arr=predictions)
#             return majority_vote
#         elif self.voting == 'soft':
#             probas = self.predict_proba(X)
#             return np.argmax(probas, axis=1)

#     def predict_proba(self, X):
#         if self.voting == 'hard':
#             raise AttributeError("predict_proba is not available when voting='hard'")
#         else:
#             probas = np.zeros((X.shape[0], 2))
#             for _, estimator in self.estimators:
#                 probas += estimator.predict_proba(X)
#             probas /= self.n_estimators
#             return probas

#     def score(self, X, y):
#         y_pred = self.predict(X)
#         return accuracy_score(y, y_pred)


In [ ]:
# result[0].shape
# we will concatenate the "Gender" and "Age" columns with the train, and test results
# df['Age_encoded'] = df['Age'].astype('category').cat.codes
# df = pd.get_dummies(df, columns=['Age'], prefix='cat')
# added_cols = [col for col in df.columns if col.startswith('is')]
# df['Age_normalized'] = (df['Age'] - df['Age'].mean()) / df['Age'].std()
# added_cols = ['Age_normalized']
# added_cols.append('Gender')
added_cols = ['Gender', 'Age_float'] + [col for col in df.columns if col.startswith('cat')]

# added_cols = []
nums = 3
for i in range(nums):
    added_cols += ['Gender', 'Age_float']
    if i % 3 == 0:
        added_cols += [col for col in df.columns if col.startswith('cat')]
# for _ in range(nums):
#     added_cols += ['Age']
#     # added_cols += ['One_word_ans_enc']
# for _ in range(nums//6):

# random.shuffle(added_cols)
# added_cols.append('One_word_ans_enc')

# df[f'{col_name}_encoded'] = df[col_name].astype('category').cat.codes
col_data = df[added_cols].values
# map the col_data to have the same shape as the result[0] using nn.Linear
# same_mapper_layer = nn.Sequential(nn.Linear(col_data.shape[1], result[0].shape[1]), nn.ReLU())
# same_mapper_layer.cuda(0)
# same_mapper_layer_test = nn.Sequential(nn.Linear(test_col_data.shape[1], test_result[0].shape[1]), nn.ReLU())
# same_mapper_layer_test.cuda(1)
# # convert them to tensor
col_data = col_data.astype(float)
col_data = torch.tensor(col_data, dtype=torch.float32).cuda(0)

# col_data = same_mapper_layer(col_data)
# test_col_data = same_mapper_layer_test(test_col_data)
# concatenate it with the tensor
result_added = torch.cat([result, col_data], dim=1)

In [ ]:
print(col_data.shape)
print(result_added.shape)
print(added_cols)

torch.Size([7200, 100])
torch.Size([7200, 868])
['Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', '

In [ ]:
# forward datasets
# class MyDataset(Dataset):
#     def __init__(self, data):
#         self.data = data
#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         return self.data[idx]
    
# ds = MyDataset(result[0])
# dl = DataLoader(ds, shuffle=False, batch_size=)

# Load each model

In [ ]:
# ie_path = './ckpt/IE/epoch_81.pt'
# sn_path = './ckpt/SN/epoch_30.pt'
# tf_path = './ckpt/TF/epoch0.pt'
# jp_path = './ckpt/jp/epoch_25.pt'
# test_number = 19
# # import csv
# from torch import nn 
MBTI = ['IE', 'SN', 'TF', 'JP']
# chkpt_path = [f'./ckpt_exp{test_number}/{name}/best.pt' for name in MBTI]
predictions = list()
mlp = nn.Sequential(nn.Linear(col_data.shape[1], 786//2),
                          nn.ReLU(),        
                          nn.Linear(786//2, 786),
                          )

import joblib
result_dir = '/root/QIA2023_phase1'
# for i in range(4):
mlp = mlp.cuda(0)

test_number = 204 # might be changed according to the unique number we want to save
mlp.load_state_dict(torch.load(f'{result_dir}/{MBTI[0]}_model_{test_number}.pth'))
mlp_output_train = mlp(col_data)
concatenated_features = np.concatenate([mlp_output_train.detach().cpu().numpy(), result[0].detach().cpu().numpy()], axis=1)
clf = joblib.load(f'{result_dir}/{MBTI[0]}_svm_{test_number}.pkl')
prob_predic = clf.predict_proba(concatenated_features)[:, 1]
predictions.append(prob_predic)

test_number = 200
mlp.load_state_dict(torch.load(f'{result_dir}/{MBTI[1]}_model_{test_number}.pth'))
mlp_output_train = mlp(col_data)
concatenated_features = np.concatenate([mlp_output_train.detach().cpu().numpy(), result[0].detach().cpu().numpy()], axis=1)
clf = joblib.load(f'{result_dir}/{MBTI[1]}_svm_{test_number}.pkl')
prob_predic = clf.predict_proba(concatenated_features)[:, 1]
predictions.append(prob_predic)

test_number = 201
mlp.load_state_dict(torch.load(f'{result_dir}/{MBTI[2]}_model_{test_number}.pth'))
mlp_output_train = mlp(col_data)
concatenated_features = np.concatenate([mlp_output_train.detach().cpu().numpy(), result[0].detach().cpu().numpy()], axis=1)
clf = joblib.load(f'{result_dir}/{MBTI[2]}_svm_{test_number}.pkl')
prob_predic = clf.predict_proba(concatenated_features)[:, 1]
predictions.append(prob_predic)
    
test_number = 200
mlp.load_state_dict(torch.load(f'{result_dir}/{MBTI[3]}_model_{test_number}.pth'))
mlp_output_train = mlp(col_data)
concatenated_features = np.concatenate([mlp_output_train.detach().cpu().numpy(), result[0].detach().cpu().numpy()], axis=1)
clf = joblib.load(f'{result_dir}/{MBTI[3]}_svm_{test_number}.pkl')
prob_predic = clf.predict_proba(concatenated_features)[:, 1]
predictions.append(prob_predic)
    
# print(chkpt_path)
# path = [ie_path, sn_path, tf_path, jp_path]

In [ ]:
# model = torch.load(ie_path, map_location='cpu')
len(predictions[0]), len(predictions[1]), len(predictions[2]), len(predictions[3])


In [ ]:
# ie, sn, tf, jp = [main(path) for path in chkpt_path]

In [ ]:
test = pd.DataFrame({
                    'I/E': predictions[0].tolist(), 
                    'S/N': predictions[1].tolist(), 
                    'T/F': predictions[2].tolist(), 
                    'J/P': predictions[3].tolist()})

In [ ]:
# test.head()
test

In [ ]:
test['idx'] = test.index + 1
test.set_index('idx', inplace=True)
test
# test_max

In [ ]:
# if you want to assign another unique number, change the sub_number
# sub_number_av = 'phase2_av_svm_only_full_data_trial1'
# sub_number_av = 'phase2_av_knn_only_full_data_trial1'
# sub_number_av = 'phase2_av_knn60pt47_only_full_data'
# # sub_number_max = 'phase2_max'
# csv_path_av = f'./submission_exp{sub_number_av}.csv'
sub_number = 'phase1'
csv_path = f'submission_exp{sub_number}.csv'
# csv_path_max = f'./submission_exp{sub_number_max}.csv'
# add the column named idx and start from '1'

test.to_csv(csv_path)
# print('saved to ', csv_path)
# test_av.to_csv(csv_path_av)
# test_max.to_csv(csv_path_max)